<a href="https://colab.research.google.com/github/robert-myers/myanimelist-recommender/blob/master/notebooks/try_it_out.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install jikanpy && pip install surprise

     |████████████████████████████████| 1.2MB 3.8MB/s 
     |████████████████████████████████| 153kB 16.3MB/s 
     |████████████████████████████████| 256kB 16.8MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=2714b661322dbe6f8c77a0774ef249bf2764d3054210bb749b6664859e83d730
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl
     |████████████████████████████████| 6.5MB 2.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678243 sha256=e90595c0c368c317646e88f093b3338ff4ce5cda098d953d52438870021b97c1
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
import pandas as pd
import string
import time

from jikanpy import Jikan
from surprise import accuracy
from surprise.dump import load

In [0]:
jikan = Jikan()

In [0]:
users = pd.read_csv("https://s3.us-east-2.amazonaws.com/my.anime.list.sagemaker/myanimelist/UserList.csv")
users = users[["username"]]
users = users.dropna()
users["lower"] = users["username"].apply(lambda x: x.lower())
users = users.set_index("lower")

In [0]:
usernames = dict(users.T)

In [0]:
def get_uid(username):
  username = username.lower()
  if username in usernames:
    return usernames[username][0]
  else:
    return username

In [0]:
anime = pd.read_csv("https://s3.us-east-2.amazonaws.com/my.anime.list.sagemaker/myanimelist/AnimeList.csv", index_col="anime_id")[["title",	"image_url"]]

In [0]:
surprised = load("/content/drive/My Drive/svd.pickle")

In [0]:
algo = surprised[1]
predictions = surprised[0]

In [12]:
accuracy.rmse(predictions)

RMSE: 1.1859


1.1859182533000143

In [13]:
"RX-78-2_Gundam".translate(str.maketrans("", "", string.punctuation))

'RX782Gundam'

In [0]:
def get_recommendations(user, num_titles=10, filter_completed=True):
  user = get_uid(user)  
  user_predictions = anime.reset_index()[["anime_id"]]
  if filter_completed:
    user_completed = pd.DataFrame(jikan.user(username=user, request='animelist', argument='completed', page=1)["anime"])
    i = 2
    downloading = True
    while downloading:
      time.sleep(4) # API requires 4 second delay between requests
      downloading = jikan.user(username=user, request='animelist', argument='completed', page=i)["anime"]
      user_completed = user_completed.append(pd.DataFrame(downloading))
      i += 1
    user_completed_lst = list(user_completed["mal_id"])
    user_predictions["est"] = user_predictions["anime_id"].apply(lambda item: algo.predict(user, item)[3] if item not in user_completed_lst else 0)
  else:
    user_predictions["est"] = user_predictions["anime_id"].apply(lambda item: algo.predict(user, item)[3])
  return user_predictions.sort_values("est", ascending=False).head(num_titles)["anime_id"].apply(lambda x: anime.loc[x])

In [15]:
%%time
get_recommendations("orionite", filter_completed=True)

CPU times: user 272 ms, sys: 0 ns, total: 272 ms
Wall time: 15 s


,title,image_url
7483,Redline,https://myanimelist.cdn-dena.com/images/anime/...
4687,Detroit Metal City,https://myanimelist.cdn-dena.com/images/anime/...
3677,Prison School,https://myanimelist.cdn-dena.com/images/anime/...
10598,Daicon Opening Animations,https://myanimelist.cdn-dena.com/images/anime/...
10810,JoJo no Kimyou na Bouken (TV),https://myanimelist.cdn-dena.com/images/anime/...
13232,Nihon Animator Mihonichi,https://myanimelist.cdn-dena.com/images/anime/...
9889,Great Teacher Onizuka,https://myanimelist.cdn-dena.com/images/anime/...
6687,Mahou Shoujo Lyrical Nanoha: The Movie 2nd A&#...,https://myanimelist.cdn-dena.com/images/anime/...
5005,Hellsing Ultimate,https://myanimelist.cdn-dena.com/images/anime/...
9880,Uchuu Senkan Yamato 2199,https://myanimelist.cdn-dena.com/images/anime/...


In [16]:
%%time
get_recommendations("orionite", num_titles=5, filter_completed=False)

CPU times: user 128 ms, sys: 4.73 ms, total: 133 ms
Wall time: 130 ms


,title,image_url
6040,FLCL,https://myanimelist.cdn-dena.com/images/anime/...
5257,Suzumiya Haruhi no Yuuutsu,https://myanimelist.cdn-dena.com/images/anime/...
11361,Byousoku 5 Centimeter,https://myanimelist.cdn-dena.com/images/anime/...
10918,Tengen Toppa Gurren Lagann,https://myanimelist.cdn-dena.com/images/anime/...
13587,Carnival Phantasm,https://myanimelist.cdn-dena.com/images/anime/...


In [0]:
users = pd.DataFrame(usernames).T.reset_index()[["index"]]

In [0]:
def get_fans(item, num_users=10, get_estimates=True):
  item_predictions = users.copy()
  item_predictions["est"] = item_predictions["index"].apply(lambda user: algo.predict(get_uid(user), item)[3])
  if get_estimates:
    return item_predictions.sort_values("est", ascending=False).head(num_users)
  else:
    return item_predictions.sort_values("est", ascending=False).head(num_users)[["index"]]

In [40]:
get_fans(227, num_users=10)

,index,est
131741,calamitymeg,10.0
138819,mellon85,10.0
138768,sindhara,10.0
138782,zindrel,10.0
21356,sasohina,10.0
21355,8bitmare,10.0
138787,solid132,10.0
138792,boogiepopxo,10.0
280182,fima04,10.0
138818,obakami,10.0
